# Run your first MESA HMS-HMS binary simulation with POSYDON

If you haven't done it already, export all the relevant POSYDON and MESA environment variables.

Tip: create a file called `.bash_POSYDON_MESA` with the following content (please adapt the PATHs to your system):
```bash
export PATH_TO_POSYDON=~/software/POSYDON/
export PATH_TO_POSYDON_DATA=~/software/POSYDON/data/

export MESA_DIR=~/software/mesa-r11701_witheoschange_andwithreverseMTchange_fiximplicitmdot
export OMP_NUM_THREADS=4
export MESASDK_ROOT=~/software/mesasdk
source $MESASDK_ROOT/bin/mesasdk_init.sh
```
so that every time you open a new terminal you can just run:
```bash
source .bash_POSYDON_MESA
```

If you haven't done it already, please download the `development` branch of the `POSYDON-MESA-INLISTS` submodule, see [Architecting MESA Simulation Grids with POSYDON](running-grids.rst). This is the branch that contains the simulation properties of all POSYDON MESA grids.

## Creating the Initialization File for the POSYDON MESA Submission Script

You can now copy in your working directory the API ini file configuration of HMS-HMS binaries, which is stored on Github as part of the POSYDON repository. The cell below will download the file for you. 

Note: in the `POSYDON-MESA-INLISTS` submodule we have more example ini files at `POSYDON-MESA_INLISTS/r11701/running_scripts/`. For each grid we support two ini file configured to the Northwestern and UNIGE HPC clusters.

In [ ]:
import urllib.request

# Download the grid_params.ini file from GitHub
url = "https://raw.githubusercontent.com/POSYDON-code/POSYDON/refs/heads/main/grid_params/grid_params.ini"
urllib.request.urlretrieve(url, './HMS-HMS.ini')
print("Download completerd")

Open the file and edit the following parameters:
- `user=your_username (can be skipped if not run via SLURM)`
- `partition=the_slurm_partition_to_run_on` (can be skipped if not run via SLURM)
- `account=your_slurm_account` (can be skipped if not run via SLURM)
- `email=your_email` (can be skipped if not run via SLURM)
- `posydon_github_root=your_path_to_POSYDON`
- `grid=your_grid_csv_file` (will get created in the next section)

The `scenario` parameter of the ini file is the main parameter that controls the simulation setup. This parameter let's us decide which simulation we want to run. The syntax is the following:
- the first argument is the name of the submodule `posydon` which points to the POSYDON-MESA-INLISTS submodule (other options are available, e.g. `user` which points to a private submodule)
- the second argument is the name of the branch and commit of the submodule we want to use connected by a dash `-`. In this case we want to use the `development` branch of the submodule and it's latest commit, so we write `development-c1f75acacc3bd645197c5d3e920a121e94d95a80`
- the third argument is the name of the simulation grid we want to run, in this case we want to run a HMS-HMS grid. The name of the grid is `HMS-HMS`.

To summarize, you should have `scenario = ['posydon', 'development-c1f75acacc3bd645197c5d3e920a121e94d95a80', 'HMS-HMS']`

The `zams_file` points to the ZAMS model used to generate the HMS stars in the binary system. POSYDON v2.0.0 supports 8 different metallicities, here we would like to use the 0.1Zsun POSYDON MESA ZAMS model `${posydon_github_root}/grid_params/POSYDON-MESA-INLISTS/r11701/ZAMS_models/zams_z1.42m3_y0.2511.data`

The last parameter of this file is `grid` and points to the `csv` file containing the initial conditions of the grid. In this case we want to run a `HMS-HMS` grid consisting of one system, so we set `grid = grid_test.csv` and generate such file specifying the metallicity, the two star masses and the initial orbital period.

## Creating the Initial Simulation Points CSV File

Note: a notebook showing you how all the grid points of v2.0.0 are generated is available in the `POSYDON-MESA-INLISTS` submodule at `$PATH_TO_POSYDON/grid_params/POSYDON-MESA-INLISTS/r11701/running_scripts/parameter_space_v2/create_csv.ipynb`

In [ ]:
import csv

with open('./grid_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
    writer.writerow([0.00142, 0.00142, 30., 21., 10.])

The above script will create a .csv file for the parameters of a MESA 'grid' with one row, i.e. practically with one binary system.
You should now have the following files in your working directory:

In [ ]:
!ls

Let's now use POSYDON to set up the MESA simulation. In your terminal run

```bash
posydon-setup-grid --grid-type fixed --inifile HMS-HMS.ini --submission-type slurm
```

The following files will be created:

In [ ]:
!ls

## Running the MESA Model and Exploring the Simulation Output

You are now ready to submit the simulation to the cluster with the following command:

In [ ]:
!./run_grid.sh

In [ ]:
!squeue -u kruckow

After the job is finished, we can check the output files:

In [ ]:
!ls

In [ ]:
!ls Zbase_0.0014_m1_30.0000_m2_21.0000_initial_z_1.4200e-03_initial_period_in_days_1.0000e+01_grid_index_0

In [ ]:
!gzip -cd Zbase_0.0014_m1_30.0000_m2_21.0000_initial_z_1.4200e-03_initial_period_in_days_1.0000e+01_grid_index_0/out.txt.gz | head -n 320

The above output is standard output of the beginning of a MESA binary run.
Congratulations, you now know how to leverage POSYDON to run a grid of MESA simulations on a computer cluster!